# Splunk App for Data Science and Deep Learning - Notebook for Autoencoder with TensorFlow and Keras (version 2.14)

## Autoencoder Example
This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk App for Data Science and Deep Learning Toolkit. As an example we use a custom autoencoder built on keras and tensorflow.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# mltkc_import
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

2024-02-01 12:17:06.097540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("TensorFlow version: " + tf.__version__)
print("Keras version: " + keras.__version__)

numpy version: 1.25.2
pandas version: 2.1.1
TensorFlow version: 2.14.0
Keras version: 2.14.0


## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| inputlookup iris.csv <br>| fit MLTKContainer algo=autoencoder mode=stage epochs=100 batch_size=4 components=2 petal_length petal_width sepal_length sepal_width into app:iris_autoencoder

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("my_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [3]:
# mltkc_stage
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [4]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("iris_autoencoder")
print(df[0:1])
print(df.shape)
print(str(param))

   sepal_length  sepal_width  petal_length  petal_width
0           5.1          3.5           1.4          0.2
(150, 4)
{'options': {'params': {'algo': 'autoencoder', 'mode': 'stage', 'epochs': '100', 'batch_size': '4', 'components': '2'}, 'args': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'feature_variables': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width'], 'model_name': 'iris_autoencoder', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '10000000', 'max_memory_usage_mb': '16000', 'max_model_size_mb': '3000', 'max_score_time': '6000', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['petal_length', 'petal_width', 'sepal_length', 'sepal_width']}


## Stage 2 - create and initialize a model

In [5]:
# mltkc_init
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    X = df[param['feature_variables']]
    print("FIT build model with input shape " + str(X.shape))
    components = 3
    activation_fn = 'relu'
    # learning_rate = 0.001
    # epsilon=0.00001 # default 1e-07
    if 'options' in param:
        if 'params' in param['options']:
            if 'components' in param['options']['params']:
                components = int(param['options']['params']['components'])
            if 'activation_func' in param['options']['params']:
                activation_fn = param['options']['params']['activation_func']
    input_shape = int(X.shape[1])
    encoder_layer = keras.layers.Dense(components, input_dim=input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    decoder_layer = keras.layers.Dense(input_shape, activation=activation_fn, kernel_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None), bias_initializer=keras.initializers.RandomUniform(minval=0, maxval=1, seed=None))
    model = keras.Sequential()
    model.add(encoder_layer)
    model.add(decoder_layer)
    #opt = keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon)
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

In [6]:
# test mltkc_stage_create_model
model = init(df,param)
print(model.summary())

FIT build model with input shape (150, 4)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2)                 10        
                                                                 
 dense_1 (Dense)             (None, 4)                 12        
                                                                 
Total params: 22 (88.00 Byte)
Trainable params: 22 (88.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


## Stage 3 - fit the model

In [7]:
# mltkc_stage_create_model_fit
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    X = df[param['feature_variables']]
    model_epochs = 100
    model_batch_size = 32
    if 'options' in param:
        if 'params' in param['options']:
            if 'epochs' in param['options']['params']:
                model_epochs = int(param['options']['params']['epochs'])
            if 'batch_size' in param['options']['params']:
                model_batch_size = int(param['options']['params']['batch_size'])
    # connect model training to tensorboard
    log_dir="/srv/notebooks/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    # run the training
    returns['fit_history'] = model.fit(x=X,
                                       y=X, 
                                       verbose=2, 
                                       epochs=model_epochs, 
                                       batch_size=model_batch_size, 
                                       #validation_data=(X, Y),
                                       callbacks=[tensorboard_callback])
    # memorize parameters
    returns['model_epochs'] = model_epochs
    returns['model_batch_size'] = model_batch_size
    returns['model_loss_acc'] = model.evaluate(x = X, y = X)
    return returns

In [8]:
returns = fit(model,df,param)
print(returns['model_loss_acc'])

Epoch 1/100
38/38 - 1s - loss: 16.8177 - accuracy: 0.6867 - 607ms/epoch - 16ms/step
Epoch 2/100
38/38 - 0s - loss: 10.9632 - accuracy: 0.7533 - 103ms/epoch - 3ms/step
Epoch 3/100
38/38 - 0s - loss: 7.4068 - accuracy: 0.8667 - 93ms/epoch - 2ms/step
Epoch 4/100
38/38 - 0s - loss: 5.2700 - accuracy: 1.0000 - 79ms/epoch - 2ms/step
Epoch 5/100
38/38 - 0s - loss: 3.9022 - accuracy: 1.0000 - 78ms/epoch - 2ms/step
Epoch 6/100
38/38 - 0s - loss: 3.0000 - accuracy: 1.0000 - 90ms/epoch - 2ms/step
Epoch 7/100
38/38 - 0s - loss: 2.3720 - accuracy: 1.0000 - 79ms/epoch - 2ms/step
Epoch 8/100
38/38 - 0s - loss: 1.9257 - accuracy: 1.0000 - 72ms/epoch - 2ms/step
Epoch 9/100
38/38 - 0s - loss: 1.5959 - accuracy: 1.0000 - 76ms/epoch - 2ms/step
Epoch 10/100
38/38 - 0s - loss: 1.3484 - accuracy: 1.0000 - 76ms/epoch - 2ms/step
Epoch 11/100
38/38 - 0s - loss: 1.1566 - accuracy: 1.0000 - 80ms/epoch - 2ms/step
Epoch 12/100
38/38 - 0s - loss: 1.0096 - accuracy: 1.0000 - 80ms/epoch - 2ms/step
Epoch 13/100
38/38 -

## Stage 4 - apply the model

In [9]:
# mltkc_stage_create_model_apply
def apply(model,df,param):
    X = df[param['feature_variables']]
    reconstruction = model.predict(x = X)
    intermediate_layer_model = keras.Model(inputs=model.input, outputs=model.layers[0].output)
    hidden = intermediate_layer_model.predict(x = X)
    y_hat = pd.concat([pd.DataFrame(reconstruction).add_prefix("reconstruction_"), pd.DataFrame(hidden).add_prefix("hidden_")], axis=1)
    return y_hat

In [10]:
# test mltkc_stage_create_model_apply
y_hat = apply(model,df,param)
print(y_hat)

5/5 [==============================] - 0s 2ms/step
     reconstruction_0  reconstruction_1  reconstruction_2  reconstruction_3  \
0            1.400967          0.146078          5.228847          2.983863   
1            1.281866          0.151235          4.887046          2.829308   
2            1.302431          0.163738          4.875044          2.821091   
3            1.465125          0.260453          4.791732          2.761806   
4            1.454677          0.173230          5.226667          2.976737   
..                ...               ...               ...               ...   
145          5.430978          1.883986          6.653288          3.229619   
146          4.946011          1.749302          6.087059          3.006009   
147          5.297531          1.851875          6.471234          3.155190   
148          5.840867          2.096334          6.609460          3.161872   
149          5.221343          1.882612          6.107088          2.984815   



## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # save keras model to hdf5 file
    # https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models
    model.save(MODEL_DIRECTORY + name + ".keras")
    return model

In [12]:
save(model,"autoencoder_iris_autoencoder")

## Stage 6 - load the model

In [13]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    model = keras.models.load_model(MODEL_DIRECTORY + name + ".keras")
    return model

In [14]:
model2 = load("autoencoder_iris_autoencoder")
model2

## Stage 7 - provide a summary of the model

In [15]:
# return model summary
def summary(model=None):
    returns = {"version": {"tensorflow": tf.__version__, "keras": keras.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        model.summary(print_fn=lambda x: s.append(x+'\n'))
        returns["summary"] = ''.join(s)
    return returns

In [16]:
summary(model)

{'version': {'tensorflow': '2.14.0', 'keras': '2.14.0'},
 'summary': 'Model: "sequential"\n_________________________________________________________________\n Layer (type)                Output Shape              Param #   \n=================================================================\n dense (Dense)               (None, 2)                 10        \n                                                                 \n dense_1 (Dense)             (None, 4)                 12        \n                                                                 \n=================================================================\nTotal params: 22 (88.00 Byte)\nTrainable params: 22 (88.00 Byte)\nNon-trainable params: 0 (0.00 Byte)\n_________________________________________________________________\n'}

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code